# texas-data-set analysis

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
spark = SparkSession.builder.appName("texas-data-eda").getOrCreate()

In [4]:
texasData = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', delimiter='\t').load("../data/PUDF_base_all_tab.txt")

In [5]:
# Partitioning/Messing around with texasData
texasData = texasData.withColumn("LENGTH_OF_STAY", texasData["LENGTH_OF_STAY"].cast(DoubleType()))
(subset, subset2, rest) = texasData.randomSplit(seed=123, weights=[.0000003, .0000003, 0.9999994]) # 18158133 total rows
samplePoints = texasData.take(5)
samplePointsRDD = sc.parallelize(samplePoints)
examplePoint = samplePoints[0]
print(examplePoint)
print(examplePoint.discharge_qtr) # accessing a field of a Row
print(type(examplePoint))
print(texasData.columns)
print(type(examplePoint['LENGTH_OF_STAY']))

Row(discharge_qtr=u'2004Q1', thcic_id=u'000100', PROVIDER_NAME=u'Austin State Hospital', FAC_TEACHING_IND=None, FAC_PSYCH_IND=u'X', FAC_REHAB_IND=None, FAC_ACUTE_CARE_IND=None, FAC_SNF_IND=None, FAC_LONG_TERM_AC_IND=None, FAC_OTHER_LTC_IND=None, FAC_PEDS_IND=None, SPEC_UNIT_1=None, SPEC_UNIT_2=None, SPEC_UNIT_3=None, SPEC_UNIT_4=None, SPEC_UNIT_5=None, ENCOUNTER_INDICATOR=u'01', SEX_CODE=u'F', TYPE_OF_ADMISSION=u'2', SOURCE_OF_ADMISSION=u'8', PAT_STATE=u'TX', PAT_ZIP=u'78612', PAT_COUNTRY=None, county=u'021', public_health_region=u'07', ADMIT_WEEKDAY=u'6', LENGTH_OF_STAY=2.0, pat_age=u'07', PAT_STATUS=u'01', RACE=u'4', ETHNICITY=u'2', FIRST_PAYMENT_SRC=u'**', SECONDARY_PAYMENT_SRC=None, TYPE_OF_BILL=u'111', private_amount=u'0.00', semi_private_amount=u'908.00', ward_amount=u'0.00', icu_amount=u'0.00', ccu_amount=u'0.00', other_amount=u'0.00', pharm_amount=u'0.00', medsurg_amount=u'0.00', dme_amount=u'0.00', used_dme_amount=u'0.00', pt_amount=u'0.00', ot_amount=u'0.00', speech_amount=u'

In [4]:
# pyspark.sql.types.Row 
r = Row({"hello" : "world", "foo" : "bar"})
r = Row(name = "Tim", age = 5, is_subscribed = False)

# Two different ways of accessing the fields of a Row

# 1 
print(type(r))
print(r.__fields__)
# 2
attributes = list(r.asDict())
attributes.remove('age')
print(attributes)

<class 'pyspark.sql.types.Row'>
['age', 'is_subscribed', 'name']
['is_subscribed', 'name']


In [7]:
# RDD Viewer
from operator import add 
def g(x): 
    print(x)
samplePointsRDD.foreach(g)

In [8]:
# Parse Function for RDD Mapping
def parsePoint(row):
    features = list(row.asDict())
    features.remove('discharge_qtr')
    return LabeledPoint(row['discharge_qtr'], features)

parsedSamplePointsRDD = samplePointsRDD.map(parsePoint)

In [5]:
# Category Configuration Example for Training
ignore = []
stringifier = StringIndexer(inputCol = "PROVIDER_NAME", outputCol = "pnIndex")
oneHotter = OneHotEncoder(inputCol = "pnIndex", outputCol = "pnVec")
vectorizer = VectorAssembler(inputCols = ["pnVec"], outputCol = "features")
assembler = VectorAssembler(inputCols = ["pnVec"], outputCol = "features")

glr = GeneralizedLinearRegression(labelCol = "LENGTH_OF_STAY", family = "gaussian", link = "identity", maxIter = 10, regParam = .3)

pipeline = Pipeline(stages = [stringifier, oneHotter, vectorizer, glr])

# create model
model = pipeline.fit(subset)

# make predictions using model 
predictions = model.transform(subset2)


In [27]:
predictions.select("prediction", "LENGTH_OF_STAY", "features").show(1)
# print coefficients of regression model
# lrm = model.stages[-1]
# print(lrm.coefficients)

Py4JJavaError: An error occurred while calling o488.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 27 in stage 30.0 failed 1 times, most recent failure: Lost task 27.0 in stage 30.0 (TID 868, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:234)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:827)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsI

In [23]:
'features' in predictions.columns

True

In [25]:

eval1 = RegressionEvaluator(labelCol = "LENGTH_OF_STAY", predictionCol = "prediction", metricName = "rmse")
eval2 = RegressionEvaluator(labelCol = "LENGTH_OF_STAY", predictionCol = "prediction", metricName = "r2")
#rmse = eval1.evaluate(predictions)
r2 = eval2.evaluate(predictions)

# Compute test error using model.summary

# trainsummary = model.summary
"""
print("Iterations: %d" % trainsummary.totalIterations)
print("\n")
print("RMSE: %d" % trainsummary.rootMeanSquaredError)
print("\n")
print("R^2: %d" & trainsummary.r2)
print("\n")
print("R^2 is {}".format(trainSummary.r2))"""

Py4JJavaError: An error occurred while calling o384.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 52 in stage 22.0 failed 1 times, most recent failure: Lost task 52.0 in stage 22.0 (TID 746, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$class.foreach(Iterator.